In [1]:
import sys
sys.path.insert(0, 'caffe/python')
import caffe
import numpy as np
import scipy.misc
import cv2
import scipy.io
import os, sys, argparse
from os.path import join, splitext, split, isfile
import pylayer

In [2]:
caffe.set_mode_gpu()

def convert(data):
    assert data.ndim == 3
    data -= np.array((104.00698793,116.66876762,122.67891434))
    data = data.transpose((2, 0, 1))
    return data

def inverse_convert(data):
    assert data.ndim == 3
    data = data.transpose((1, 2, 0))
    data += np.array((104.00698793,116.66876762,122.67891434))
    return data

def rescale_image(image):
    return image / 255.
    
def convert_label(label):
    mask = label >= 25
    label[mask] = 1
    label[~mask] = 0
    return label

def forward(data, label):
    net.blobs['data'].reshape(1, *data.shape)
    net.blobs['data'].data[...] = data
    net.blobs['label'].reshape(1, 1, *label.shape)
    net.blobs['label'].data[...] = label
    return net.forward()




argnet = 'model/hed_train_attack.pt'

model = 'model/hed_pretrained_bsds.caffemodel'

net = caffe.Net(argnet, model, caffe.TRAIN)


test_dir = 'data/HED-BSDS/test' # test images directory


In [3]:

from scipy.ndimage.filters import gaussian_filter

def get_image(image_name):
    
    img = cv2.cvtColor(cv2.imread(image_name), cv2.COLOR_BGR2RGB).astype(np.float32)
    return convert(img)
def get_label(label_name):
    img = cv2.imread(label_name).astype(np.float32)
    return convert_label(np.squeeze(img[:,:,:1]))

#perturbation as per https://arxiv.org/abs/1803.06978
def perturb_image(image):
    
    if np.random.uniform() < 0.0:

        _, height, width = image.shape

        resize_dims = (300, 300)

        rheight = np.random.randint(resize_dims[0], image.shape[1])
        rwidth = np.random.randint(resize_dims[1], image.shape[2])

        image = scipy.misc.imresize(image, (min(height, rheight), min(width, rwidth)))

        dheight = height - image.shape[0]
        dwidth = width - image.shape[1]

        pad_top = np.random.randint(dheight)
        pad_left = np.random.randint(dwidth)

        image = np.pad(image, ((pad_top, dheight - pad_top), (pad_left, dwidth - pad_left), (0,0)), 'constant')
        image = image.transpose(2, 0, 1)
        
    return image
    
def attack(data, label, targeted=False, suppress=False):
    step = 5
    epsilon = 256
    delta = np.zeros_like(data)
    accum = np.zeros_like(delta)
    mom = 0.5
    for i in xrange(50):
#         if i % 10 == 0:
#             l *= np.where(np.random.randint(10) > 3, 1, 0)
        forward(np.clip(perturb_image(data) + delta, 0, 255), label)
        net.backward(diffs=['data'])
        grad = np.squeeze(net.blobs['data'].diff)
        
        accum = mom * accum + grad
        
        delta += (-1 if targeted else 1) * step * np.sign(accum)
        delta = np.clip(delta, -epsilon, epsilon)
        
        
    return np.clip(data + delta, 0, 255)

im = get_image(os.path.join(test_dir, os.listdir(test_dir)[0]))

lab = get_label('./ccvl.png')

attacked = attack(im, lab, True)




diff = np.mean(attacked - im, axis=0)
plt.imshow(diff)

plt.imshow(rescale_image(inverse_convert(attacked)))

edge = forward_test(attacked)

plt.imshow(np.squeeze(edge))

In [4]:
import matplotlib.pyplot as plt

In [5]:
import scipy

labs = {}
for lab in os.listdir('../BSR/BSDS500/data/groundTruth/test'):
    gt = convert_label(255 * np.mean([x[0][0][1] for x in scipy.io.loadmat('../BSR/BSDS500/data/groundTruth/test/{}'.format(lab))['groundTruth'][0]], axis=0))
    name = lab.split('.')[0]

    labs[name] = gt
    
attacks = {}
for image in os.listdir(test_dir):
    name = image.split('.')[0]
    att = attack(get_image(os.path.join(test_dir, image)), np.zeros_like(labs[name]), True, True)
#     att = attack(get_image(os.path.join(test_dir, image)), labs[name], False)
    
    attacks[image] = att
    print('attacked ', image)

('attacked ', '157087.jpg')


KeyboardInterrupt: 

In [6]:

def forward_test(dat):
    net.blobs['data'].reshape(1, *dat.shape)
    net.blobs['data'].data[...] = dat
    net.forward()
    return np.array(net.blobs['sigmoid_fuse'].data)

argnet = 'model/hed_test.pt'

net = caffe.Net(argnet, model, caffe.TEST)



In [7]:

attacks_dir = './output_attacks_suppress_256'
real_dir = './output'

count = 0
for image in attacks:
    edge = np.squeeze(forward_test(attacks[image]))
    scipy.misc.imsave(join(attacks_dir, image.split('.')[0] + '.png'), edge / edge.max())
    scipy.misc.imsave(join(attacks_dir, image.split('.')[0] + '_im.png'), attacks[image].transpose(1, 2, 0))
    
    count += 1
    print('attacked ', count)
#     edge = np.squeeze(forward_test(get_image(os.path.join(test_dir, image))))
#     scipy.misc.imsave(join(real_dir, image.split('.')[0] + '.png'), edge / edge.max())

('attacked ', 1)


/home/christian/.conda/envs/caf/lib/python2.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  
/home/christian/.conda/envs/caf/lib/python2.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  if __name__ == '__main__':
